In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/melting-point/sample_submission.csv
/kaggle/input/melting-point/train.csv
/kaggle/input/melting-point/test.csv


In [2]:
!pip install -qU rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 37.4 MB/s eta 0:00:00


In [3]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.base import clone

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# RDKit for molecular features
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, rdMolDescriptors
from rdkit.Chem import Lipinski, Crippen, MolSurf, GraphDescriptors, Fragments

import inspect


print("="*60)
print("MELTING POINT PREDICTION - IMPROVED VERSION")
print("="*60)

train = pd.read_csv('/kaggle/input/melting-point/train.csv')
test = pd.read_csv('/kaggle/input/melting-point/test.csv')

print(f"\nTrain shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"Target: Tm (melting point in Kelvin)")
print(f"Metric: MAE (Mean Absolute Error)")


def extract_comprehensive_molecular_features(smiles):
    """Extract comprehensive molecular descriptors from SMILES"""
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        return {}
    
    features = {}
    
    features['MolWt'] = Descriptors.MolWt(mol)
    features['MolLogP'] = Descriptors.MolLogP(mol)
    features['NumHDonors'] = Descriptors.NumHDonors(mol)
    features['NumHAcceptors'] = Descriptors.NumHAcceptors(mol)
    features['TPSA'] = Descriptors.TPSA(mol)
    
    features['HeavyAtomCount'] = Lipinski.HeavyAtomCount(mol)
    features['NHOHCount'] = Lipinski.NHOHCount(mol)
    features['NOCount'] = Lipinski.NOCount(mol)
    features['NumAliphaticRings'] = Lipinski.NumAliphaticRings(mol)
    features['NumAromaticRings'] = Lipinski.NumAromaticRings(mol)
    features['NumRotatableBonds'] = Lipinski.NumRotatableBonds(mol)
    features['RingCount'] = Lipinski.RingCount(mol)
    features['NumSaturatedRings'] = Lipinski.NumSaturatedRings(mol)
    
    features['MolMR'] = Crippen.MolMR(mol)  # Molar Refractivity
    
    features['LabuteASA'] = MolSurf.LabuteASA(mol)
    
    try:
        features['BalabanJ'] = GraphDescriptors.BalabanJ(mol)
    except:
        features['BalabanJ'] = 0
    features['BertzCT'] = GraphDescriptors.BertzCT(mol)
    try:
        features['Ipc'] = GraphDescriptors.Ipc(mol)
    except:
        features['Ipc'] = 0
    
    features['Kappa1'] = Descriptors.Kappa1(mol)
    features['Kappa2'] = Descriptors.Kappa2(mol)
    features['Kappa3'] = Descriptors.Kappa3(mol)
    features['HallKierAlpha'] = Descriptors.HallKierAlpha(mol)
    
    features['Chi0v'] = Descriptors.Chi0v(mol)
    features['Chi1v'] = Descriptors.Chi1v(mol)
    features['Chi2v'] = Descriptors.Chi2v(mol)
    features['Chi3v'] = Descriptors.Chi3v(mol)
    features['Chi4v'] = Descriptors.Chi4v(mol)
    
    features['NumValenceElectrons'] = Descriptors.NumValenceElectrons(mol)
    features['NumRadicalElectrons'] = Descriptors.NumRadicalElectrons(mol)
    
    try:
        features['FractionCsp3'] = Descriptors.FractionCSP3(mol)
    except:
        features['FractionCsp3'] = 0
    
    features['MaxEStateIndex'] = Descriptors.MaxEStateIndex(mol)
    features['MinEStateIndex'] = Descriptors.MinEStateIndex(mol)
    
    features['NumAtoms'] = mol.GetNumAtoms()
    features['NumHeavyAtoms'] = mol.GetNumHeavyAtoms()
    features['NumBonds'] = mol.GetNumBonds()
    features['NumHeteroatoms'] = Descriptors.NumHeteroatoms(mol)
    features['FormalCharge'] = Chem.GetFormalCharge(mol)
    
    features['NumAliphaticCarbocycles'] = Descriptors.NumAliphaticCarbocycles(mol)
    features['NumAliphaticHeterocycles'] = Descriptors.NumAliphaticHeterocycles(mol)
    features['NumAromaticCarbocycles'] = Descriptors.NumAromaticCarbocycles(mol)
    features['NumAromaticHeterocycles'] = Descriptors.NumAromaticHeterocycles(mol)
    features['NumSaturatedCarbocycles'] = Descriptors.NumSaturatedCarbocycles(mol)
    features['NumSaturatedHeterocycles'] = Descriptors.NumSaturatedHeterocycles(mol)
    
    features['fr_Al_OH'] = Fragments.fr_Al_OH(mol)
    features['fr_Ar_OH'] = Fragments.fr_Ar_OH(mol)
    features['fr_COO'] = Fragments.fr_COO(mol)
    features['fr_C_O'] = Fragments.fr_C_O(mol)
    features['fr_NH0'] = Fragments.fr_NH0(mol)
    features['fr_NH1'] = Fragments.fr_NH1(mol)
    features['fr_NH2'] = Fragments.fr_NH2(mol)
    features['fr_alkyl_halide'] = Fragments.fr_alkyl_halide(mol)
    features['fr_benzene'] = Fragments.fr_benzene(mol)
    features['fr_ester'] = Fragments.fr_ester(mol)
    features['fr_ether'] = Fragments.fr_ether(mol)
    features['fr_halogen'] = Fragments.fr_halogen(mol)
    features['fr_ketone'] = Fragments.fr_ketone(mol)
    features['fr_phenol'] = Fragments.fr_phenol(mol)
    features['fr_nitro'] = Fragments.fr_nitro(mol)
    
    # Radius 1
    fp_r1 = AllChem.GetMorganFingerprintAsBitVect(mol, radius=1, nBits=256)
    for idx, bit in enumerate(fp_r1):
        features[f'morgan_r1_{idx}'] = int(bit)
    
    # Radius 2 (most common)
    fp_r2 = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=512)
    for idx, bit in enumerate(fp_r2):
        features[f'morgan_r2_{idx}'] = int(bit)
    
    # Radius 3 (longer range)
    fp_r3 = AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=256)
    for idx, bit in enumerate(fp_r3):
        features[f'morgan_r3_{idx}'] = int(bit)
    
    return features

print("\n" + "="*60)
print("EXTRACTING MOLECULAR FEATURES")
print("="*60)
print("This will take 2-3 minutes...")

train_mol_features = train['SMILES'].apply(extract_comprehensive_molecular_features)
train_mol_df = pd.DataFrame(list(train_mol_features)).fillna(0)

test_mol_features = test['SMILES'].apply(extract_comprehensive_molecular_features)
test_mol_df = pd.DataFrame(list(test_mol_features)).fillna(0)

print(f"\n✓ Extracted {train_mol_df.shape[1]} molecular features")
print(f"  Train: {train_mol_df.shape}")
print(f"  Test: {test_mol_df.shape}")

# correlation with target
print("\nTop 15 molecular features correlated with Tm:")
correlations = train_mol_df.corrwith(train['Tm']).abs().sort_values(ascending=False)
print(correlations.head(15))


print("\n" + "="*60)
print("PREPARING FEATURES")
print("="*60)

id_cols = ['id', 'SMILES', 'Tm']
feature_cols = [col for col in train.columns if col not in id_cols]

X_group = train[feature_cols]
X_test_group = test[feature_cols]

zero_var_group = X_group.columns[X_group.std() == 0]
print(f"Removing {len(zero_var_group)} zero-variance Group features")
X_group = X_group.drop(columns=zero_var_group)
X_test_group = X_test_group.drop(columns=zero_var_group)

X_combined = pd.concat([X_group, train_mol_df], axis=1)
X_test_combined = pd.concat([X_test_group, test_mol_df], axis=1)

print(f"\nCombined features:")
print(f"  Group features: {X_group.shape[1]}")
print(f"  Molecular features: {train_mol_df.shape[1]}")
print(f"  Total: {X_combined.shape[1]}")

zero_var_combined = X_combined.columns[X_combined.std() == 0]
if len(zero_var_combined) > 0:
    print(f"  Removing {len(zero_var_combined)} zero-variance features")
    X_combined = X_combined.drop(columns=zero_var_combined)
    X_test_combined = X_test_combined.drop(columns=zero_var_combined)

print(f"  Final feature count: {X_combined.shape[1]}")

y = train['Tm']


print("\n" + "="*60)
print("FEATURE SELECTION")
print("="*60)

k_features = min(800, X_combined.shape[1])  # Adjusting based on feature count
selector = SelectKBest(f_regression, k=k_features)
X_selected = selector.fit_transform(X_combined, y)
X_test_selected = selector.transform(X_test_combined)

selected_features = X_combined.columns[selector.get_support()]
print(f"Selected {len(selected_features)} most important features")

X_combined = pd.DataFrame(X_selected, columns=selected_features, index=X_combined.index)
X_test_combined = pd.DataFrame(X_test_selected, columns=selected_features, index=X_test_combined.index)


class RegressionTrainer:
    def __init__(self, model, n_folds=5, random_state=42):
        self.model = model
        self.n_folds = n_folds
        self.random_state = random_state
        self.kfold = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
        
    def train_predict(self, X_train, y_train, X_test, verbose=True):
        oof_preds = np.zeros(len(X_train))
        test_preds = np.zeros(len(X_test))
        
        for fold, (train_idx, val_idx) in enumerate(self.kfold.split(X_train), 1):
            if verbose:
                print(f"Fold {fold}/{self.n_folds}...", end=" ")
            
            X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
            
            fold_model = clone(self.model)
            
            # Check if model supports eval_set
            if hasattr(fold_model, 'fit') and 'eval_set' in inspect.signature(fold_model.fit).parameters:
                fit_params = {'eval_set': [(X_val, y_val)]}
                
                if 'verbose' in inspect.signature(fold_model.fit).parameters:
                    fit_params['verbose'] = False
                
                fold_model.fit(X_tr, y_tr, **fit_params)
            else:
                fold_model.fit(X_tr, y_tr)
            
            # predict
            oof_preds[val_idx] = fold_model.predict(X_val)
            test_preds += fold_model.predict(X_test) / self.n_folds
            
            fold_mae = mean_absolute_error(y_val, oof_preds[val_idx])
            if verbose:
                print(f"MAE: {fold_mae:.4f}")
        
        overall_mae = mean_absolute_error(y_train, oof_preds)
        if verbose:
            print(f"{'='*50}")
            print(f"Overall OOF MAE: {overall_mae:.4f}")
            print(f"{'='*50}\n")
        
        return oof_preds, test_preds, overall_mae


print("\n" + "="*60)
print("TRAINING MODELS")
print("="*60)

# model 1: XGBoost
print("\n XGBoost")
print("-"*60)

xgb_model = XGBRegressor(
    n_estimators=4000,
    max_depth=9,
    learning_rate=0.008,
    subsample=0.85,
    colsample_bytree=0.85,
    min_child_weight=2,
    gamma=0.2,
    reg_alpha=0.5,
    reg_lambda=1.0,
    random_state=42,
)

trainer = RegressionTrainer(xgb_model, n_folds=5)
oof_xgb, test_xgb, mae_xgb = trainer.train_predict(X_combined, y, X_test_combined)

# model 2: LightGBM
print("\n LightGBM")
print("-"*60)

lgb_model = LGBMRegressor(
    n_estimators=4000,
    num_leaves=80,
    learning_rate=0.008,
    subsample=0.85,
    colsample_bytree=0.85,
    min_child_samples=15,
    reg_alpha=0.5,
    reg_lambda=1.0,
    random_state=42,
    force_row_wise=True,
    verbosity=-1
)

trainer = RegressionTrainer(lgb_model, n_folds=5)
oof_lgb, test_lgb, mae_lgb = trainer.train_predict(X_combined, y, X_test_combined)

# model 3: CatBoost
print("\n CatBoost")
print("-"*60)

cat_model = CatBoostRegressor(
    iterations=4000,
    depth=9,
    learning_rate=0.008,
    l2_leaf_reg=5,
    random_state=42,
    verbose=False
)

trainer = RegressionTrainer(cat_model, n_folds=5)
oof_cat, test_cat, mae_cat = trainer.train_predict(X_combined, y, X_test_combined)

# model 4: XGBoost variant (different hyperparameters)
print("\n XGBoost Variant")
print("-"*60)

xgb_model2 = XGBRegressor(
    n_estimators=5000,
    max_depth=7,
    learning_rate=0.005,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=4,
    gamma=0.1,
    reg_alpha=1.0,
    reg_lambda=2.0,
    random_state=123,
)

trainer = RegressionTrainer(xgb_model2, n_folds=5)
oof_xgb2, test_xgb2, mae_xgb2 = trainer.train_predict(X_combined, y, X_test_combined)

# model 5: LightGBM variant
print("\n LightGBM Variant")
print("-"*60)

lgb_model2 = LGBMRegressor(
    n_estimators=5000,
    num_leaves=50,
    learning_rate=0.005,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_samples=25,
    reg_alpha=1.0,
    reg_lambda=2.0,
    random_state=123,
    force_row_wise=True,
    verbosity=-1
)

trainer = RegressionTrainer(lgb_model2, n_folds=5)
oof_lgb2, test_lgb2, mae_lgb2 = trainer.train_predict(X_combined, y, X_test_combined)


print("\n Neural Network")
print("-"*60)

# Scale features for NN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)
X_test_scaled = scaler.transform(X_test_combined)

# Simple feedforward NN
try:
    import tensorflow as tf
    from tensorflow import keras
    
    oof_nn = np.zeros(len(X_scaled))
    test_nn = np.zeros(len(X_test_scaled))
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled), 1):
        print(f"Fold {fold}/5...", end=" ")
        
        X_tr, X_val = X_scaled[train_idx], X_scaled[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model = keras.Sequential([
            keras.layers.Dense(512, activation='relu', input_dim=X_scaled.shape[1]),
            keras.layers.BatchNormalization(),
            keras.layers.Dropout(0.3),
            
            keras.layers.Dense(256, activation='relu'),
            keras.layers.BatchNormalization(),
            keras.layers.Dropout(0.3),
            
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dropout(0.2),
            
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dropout(0.2),
            
            keras.layers.Dense(1)
        ])
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='mae'
        )
        
        early_stop = keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=50,
            restore_best_weights=True
        )
        
        model.fit(
            X_tr, y_tr,
            validation_data=(X_val, y_val),
            epochs=300,
            batch_size=32,
            callbacks=[early_stop],
            verbose=0
        )
        
        oof_nn[val_idx] = model.predict(X_val, verbose=0).ravel()
        test_nn += model.predict(X_test_scaled, verbose=0).ravel() / 5
        
        fold_mae = mean_absolute_error(y_val, oof_nn[val_idx])
        print(f"MAE: {fold_mae:.4f}")
    
    mae_nn = mean_absolute_error(y, oof_nn)
    print(f"{'='*50}")
    print(f"Overall OOF MAE: {mae_nn:.4f}")
    print(f"{'='*50}\n")
    
    has_nn = True
except:
    print("TensorFlow not available, skipping Neural Network")
    has_nn = False


print("\n" + "="*60)
print("CREATING ADVANCED ENSEMBLE")
print("="*60)

# collect all predictions
if has_nn:
    oof_stack = np.column_stack([oof_xgb, oof_lgb, oof_cat, oof_xgb2, oof_lgb2, oof_nn])
    test_stack = np.column_stack([test_xgb, test_lgb, test_cat, test_xgb2, test_lgb2, test_nn])
    model_names = ['XGBoost', 'LightGBM', 'CatBoost', 'XGBoost2', 'LightGBM2', 'NN']
else:
    oof_stack = np.column_stack([oof_xgb, oof_lgb, oof_cat, oof_xgb2, oof_lgb2])
    test_stack = np.column_stack([test_xgb, test_lgb, test_cat, test_xgb2, test_lgb2])
    model_names = ['XGBoost', 'LightGBM', 'CatBoost', 'XGBoost2', 'LightGBM2']

print(f"\nEnsembling {len(model_names)} models")

#  different meta-learners
meta_models = {
    'Ridge_0.1': Ridge(alpha=0.1),
    'Ridge_1': Ridge(alpha=1.0),
    'Ridge_10': Ridge(alpha=10.0),
    'Lasso_0.1': Lasso(alpha=0.1),
    'ElasticNet': ElasticNet(alpha=0.1),
    'LightGBM': LGBMRegressor(n_estimators=100, learning_rate=0.05, verbosity=-1)
}

best_meta_mae = float('inf')
best_meta_name = None
best_meta_model = None

print("\nTesting meta-learners:")
for name, meta_model in meta_models.items():
    meta_model.fit(oof_stack, y)
    meta_oof = meta_model.predict(oof_stack)
    mae = mean_absolute_error(y, meta_oof)
    print(f"  {name:15s}: MAE = {mae:.4f}")
    
    if mae < best_meta_mae:
        best_meta_mae = mae
        best_meta_name = name
        best_meta_model = meta_model

print(f"\n Best meta-learner: {best_meta_name}")
print(f" Best ensemble MAE: {best_meta_mae:.4f}")

# Show weights if Ridge
if 'Ridge' in best_meta_name:
    print(f"\nModel weights:")
    for name, coef in zip(model_names, best_meta_model.coef_):
        print(f"  {name:12s}: {coef:8.4f}")
    print(f"  {'Intercept':12s}: {best_meta_model.intercept_:8.4f}")

# final predictions
final_test = best_meta_model.predict(test_stack)


print("\n" + "="*60)
print("CREATING SUBMISSION")
print("="*60)

submission = pd.DataFrame({
    'id': test['id'],
    'Tm': final_test
})

submission.to_csv('submission.csv', index=False)

print(f"\n Submission created!")
print(f"\n Expected CV Score: {best_meta_mae:.4f}")
print(f" Improvement from 25.41 to {best_meta_mae:.4f}")
print(f" Improvement: {25.41 - best_meta_mae:.4f} MAE points ({((25.41 - best_meta_mae)/25.41)*100:.1f}%)")

print(f"\nSanity checks:")
print(f"  Prediction range: {final_test.min():.1f}K to {final_test.max():.1f}K")
print(f"  Training range: {y.min():.1f}K to {y.max():.1f}K")
print(f"  Mean prediction: {final_test.mean():.1f}K")
print(f"  Mean training: {y.mean():.1f}K")

print("\n File: submission.csv")
print("\n First 10 predictions:")
print(submission.head(10))


MELTING POINT PREDICTION - IMPROVED VERSION

Train shape: (2662, 427)
Test shape: (666, 426)
Target: Tm (melting point in Kelvin)
Metric: MAE (Mean Absolute Error)

EXTRACTING MOLECULAR FEATURES
This will take 2-3 minutes...


[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerator
[01:29:18] DEPRECATION WARNING: please use MorganGenerat


✓ Extracted 1082 molecular features
  Train: (2662, 1082)
  Test: (666, 1082)

Top 15 molecular features correlated with Tm:
BertzCT                   0.576550
NumBonds                  0.534738
HeavyAtomCount            0.494154
NumAtoms                  0.494154
NumHeavyAtoms             0.494154
LabuteASA                 0.483962
RingCount                 0.477688
NumAromaticRings          0.476362
MolMR                     0.460410
fr_benzene                0.454207
NumAromaticCarbocycles    0.454207
MolWt                     0.447791
morgan_r2_356             0.428590
morgan_r1_100             0.428568
morgan_r3_100             0.426252
dtype: float64

PREPARING FEATURES
Removing 87 zero-variance Group features

Combined features:
  Group features: 337
  Molecular features: 1082
  Total: 1419
  Removing 3 zero-variance features
  Final feature count: 1416

FEATURE SELECTION
Selected 800 most important features

TRAINING MODELS

 XGBoost
-------------------------------------------

2026-01-29 02:15:33.074890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769652933.305458      17 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769652933.373534      17 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769652933.953244      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769652933.953280      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769652933.953283      17 computation_placer.cc:177] computation placer alr

Fold 1/5... 

2026-01-29 02:15:50.035920: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


MAE: 30.6581
Fold 2/5... MAE: 29.0826
Fold 3/5... MAE: 29.2931
Fold 4/5... MAE: 29.7533
Fold 5/5... MAE: 29.5602
Overall OOF MAE: 29.6696


CREATING ADVANCED ENSEMBLE

Ensembling 6 models

Testing meta-learners:
  Ridge_0.1      : MAE = 27.4222
  Ridge_1        : MAE = 27.4222
  Ridge_10       : MAE = 27.4222
  Lasso_0.1      : MAE = 27.4207
  ElasticNet     : MAE = 27.4211
  LightGBM       : MAE = 23.0219

 Best meta-learner: LightGBM
 Best ensemble MAE: 23.0219

CREATING SUBMISSION

 Submission created!

 Expected CV Score: 23.0219
 Improvement from 25.41 to 23.0219
 Improvement: 2.3881 MAE points (9.4%)

Sanity checks:
  Prediction range: 118.9K to 579.9K
  Training range: 53.5K to 897.1K
  Mean prediction: 277.0K
  Mean training: 278.3K

 File: submission.csv

 First 10 predictions:
     id          Tm
0  1022  347.754253
1  1146  324.392297
2    79  214.310664
3  2279  218.364014
4  1342  237.872606
5  2082  341.365432
6    29  239.587294
7   515  320.850061
8  2309  273.413566
9 